In [ ]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')
print("PaddleOCR initialized successfully")

In [ ]:
import os
from flask import Flask, request, render_template, jsonify
from groq import Groq
from paddleocr import PaddleOCR
import csv

app = Flask(__name__)

# Initialize OCR and Groq (unchanged)
ocr = PaddleOCR(use_angle_cls=True, lang='en')
client = Groq(api_key="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")

# Global variables (unchanged)
image_path = None
csv_path = 'data.csv'
log_csv_path = 'refined_text_log.csv'

# Original logic functions (unchanged)
def log_refined_text(refined_text):
    """
    Logs the refined OCR-extracted text into a CSV file.
    """
    with open(log_csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([refined_text])

def extract_and_clean_text(image_path):
    """
    Extracts text from an image using OCR and cleans it.
    """
    results = ocr.ocr(image_path, cls=True)
    cleaned_text = []
    for line in results[0]:
        text = line[1][0]
        if len(text) >= 3 and text.isprintable():
            cleaned_text.append(text)
    return cleaned_text

def process_image_and_csv(image_path, csv_path):
    """
    Processes a nutritional image, sending data to the Groq API for analysis.
    """
    if not image_path:
        return "Error: No image path provided!", "", ""
    try:
        # Extract text from the image
        cleaned_text = extract_and_clean_text(image_path)
        extracted_text = ", ".join(cleaned_text)

        # Sending the OCR text to NLP model
        prompt = f"""
        I am using OCR to extract the Nutritional information from the Food pack labels. 
        I need you to refine the text: {extracted_text}. Just return the nutritional facts and Ingredients. 
        Based on the ingredients, what is the Food name? No other words.
        """

        # Send the prompt to Groq
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "system", "content": "You are a professional medical advisor."},
                      {"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300,
            top_p=1,
            stream=False
        )
        refined_text = response.choices[0].message.content
        log_refined_text(refined_text)
        return refined_text, "", ""
    except Exception as e:
        return f"Error occurred: {e}", "", ""

# Flask setup for file uploads
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB file size limit for safety

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload_nutritional', methods=['POST'])
def upload_nutritional():
    global image_path
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(filepath)
    image_path = filepath
    refined_text, _, _ = process_image_and_csv(image_path, csv_path)
    return jsonify({"refined_text": refined_text, "image_url": f"/{filepath}"})

@app.route('/upload_medical', methods=['POST'])
def upload_medical():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(filepath)
    extracted_text = extract_and_clean_text(filepath)
    medical_prompt = f"""
    I am using OCR to extract the text from a medical report. 
    Refine the text: {extracted_text}, remove any noise, and provide a clear summary of the medical findings. 
    Just return the important medical details and diagnosis if available, no extra words.
    """
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "system", "content": "You are a professional medical advisor."},
                  {"role": "user", "content": medical_prompt}],
        temperature=0.7,
        max_tokens=300,
        top_p=1,
        stream=False
    )
    return jsonify({"refined_text": response.choices[0].message.content})

@app.route('/evaluate_combined', methods=['POST'])
def evaluate_combined():
    data = request.json
    nutritional_text = data.get('nutritional_text', '').strip()
    medical_text = data.get('medical_text', '').strip()
    selected_model = data.get('model', 'llama-3.3-70b-versatile')
    selected_language = data.get('language', 'English')

    if not nutritional_text:
        return jsonify({"error": "Please analyze nutritional data first."}), 400
    if not medical_text:
        return jsonify({"error": "Please process medical report first."}), 400

    next_prompt = f"""
    Dear User,

    Based on the extracted text from your food pack labels: {nutritional_text},
    and the details from your medical report: {medical_text},
    please evaluate the ingredients for safety.
    Provide a short recommendation on whether the food is safe to consume,
    including the safe quantity for intake if applicable.
    If the food is not recommended, briefly explain why it should be avoided.

    Please provide the response in the following format:

    1. First, a short and clear recommendation in **English**.
    2. After that, a short and clear recommendation in **{selected_language}** that corresponds to the English response.
    """
    final_response = client.chat.completions.create(
        model=selected_model,
        messages=[{"role": "system", "content": "You are a professional medical advisor."},
                  {"role": "user", "content": next_prompt}],
        temperature=0.7,
        max_tokens=400,
        top_p=1,
        stream=False
    )
    return jsonify({"result": final_response.choices[0].message.content})

if __name__ == '__main__':
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    app.run(debug=True)

In [ ]:
import paddle
print(paddle.is_compiled_with_cuda())  # Should return True

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


In [ ]:
pip uninstall tensorflow-gpu

In [ ]:
import paddle
paddle.utils.run_check()


In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("cuDNN Version:", torch.backends.cudnn.version() if torch.cuda.is_available() else "N/A")

In [2]:
pip install flask_cors 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Srevarshan\OneDrive\Desktop\FOOD WEB\deploy\Scripts\python.exe -m pip install --upgrade pip' command.
